# GPU Evaluation with Claude API Error Handling

This notebook can automatically ask Claude for help when errors occur.

In [ ]:
# Install Anthropic SDK
!pip install -q anthropic loguru pandas pyarrow scikit-learn numpy torch

In [ ]:
# Set up Claude API (you'll need your API key)
import os
from anthropic import Anthropic

# Enter your API key here or use Colab secrets
ANTHROPIC_API_KEY = ""  # Get from https://console.anthropic.com/

# Or use Colab secrets (recommended)
# from google.colab import userdata
# ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = Anthropic(api_key=ANTHROPIC_API_KEY)

def ask_claude(question: str, error: str = None) -> str:
    """Ask Claude for help with errors."""
    prompt = f"""I'm running model evaluation in Google Colab and encountered an issue.

Question: {question}

{f'Error: {error}' if error else ''}

Please provide a concise solution (code or instructions)."""
    
    message = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=2000,
        messages=[{"role": "user", "content": prompt}]
    )
    return message.content[0].text

print("✅ Claude API configured")

In [ ]:
# Setup with error handling
import sys
import traceback

def run_with_claude_help(func, *args, **kwargs):
    """Run function with automatic Claude error handling."""
    try:
        return func(*args, **kwargs)
    except Exception as e:
        error_msg = traceback.format_exc()
        print(f"❌ Error occurred: {e}")
        print("\n🤖 Asking Claude for help...\n")
        
        solution = ask_claude(
            question=f"How do I fix this error in {func.__name__}?",
            error=error_msg
        )
        print("💡 Claude's suggestion:")
        print(solution)
        raise

print("✅ Error handling configured")

In [ ]:
# Mount Google Drive (faster for large files)
from google.colab import drive
drive.mount('/content/drive')

# Copy files from Drive
# First, upload /tmp/colab_upload/ to your Google Drive
# Then update the path below

!mkdir -p models/new data/features
!cp /content/drive/MyDrive/colab_upload/models/*.pt models/new/
!cp /content/drive/MyDrive/colab_upload/features/*.parquet data/features/

In [ ]:
# Create all necessary modules (same as before)
!mkdir -p apps/trainer/{models,train,eval} libs/{constants,rl_env}

# [Include all the %%writefile cells from the original notebook here]

In [ ]:
# Run evaluation with automatic error handling
def main_evaluation():
    # Your evaluation code here
    !python evaluate_gpu.py

run_with_claude_help(main_evaluation)